<h1>Numpy and Pandas Home Assignment</h1>
In this assignment, we will use pandas and numpy to do some fairly basic analysis on equities.

<h3>Imports</h3>

In [25]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import pandas_datareader.data as web
import yfinance as yf

<h3>Get data on the following four stocks from yahoo finance</h3>
<li>MS, GS, JPM, C
<li>Use the date range 2013/1/1/ to 2018/9/24

In [26]:
data = yf.download("MS GS JPM C", start="2013-01-01", end="2018-09-24")
data.head()

[*********************100%***********************]  4 of 4 completed


Adj Close                                        Close  \
                    C          GS        JPM         MS          C   
Date                                                                 
2013-01-02  33.872852  111.239899  33.821274  15.679015  41.250000   
2013-01-03  33.987816  110.631531  33.753094  15.647051  41.389999   
2013-01-04  34.841827  113.647835  34.351360  16.134520  42.430000   
2013-01-07  34.874676  113.436646  34.389236  15.822863  42.470001   
2013-01-08  34.866451  112.414299  34.457375  15.702986  42.459999   

                                                   High              ...  \
                    GS        JPM         MS          C          GS  ...   
Date                                                                 ...   
2013-01-02  131.660004  44.660000  19.620001  41.400002  131.660004  ...   
2013-01-03  130.940002  44.570000  19.580000  41.500000  131.779999  ...   
2013-01-04  134.509995  45.360001  20.190001  42.540001  134.639999  ...   
2013-01-07  134.259995  45.410000  19.799999  42.549999  134.509995  ...   
2013-01-08  133.050003  45.500000  19.650000  42.470001  133.949997  ...   

                  Low              Open                                    \
                  JPM     MS          C          GS        JPM         MS   
Date                                                                        
2013-01-02  44.220001  19.51  40.910000  131.300003  44.980000  20.160000   
2013-01-03  44.200001  19.32  41.169998  131.699997  44.740002  19.459999   
2013-01-04  44.439999  19.58  41.759998  130.929993  44.450001  19.680000   
2013-01-07  45.020000  19.67  42.540001  134.119995  45.090000  20.020000   
2013-01-08  44.959999  19.59  42.290001  133.710007  45.400002  19.740000   

              Volume                               
                   C       GS       JPM        MS  
Date                                               
2013-01-02  45648500  4449000  31413000  29812900  
2013-01-03  38343300  4584600  24227700  29859500  
2013-01-04  46310100  5234600  24487700  31983900  
2013-01-07  36305100  3556900  24456900  21445100  
2013-01-08  28185200  2760200  19624200  22206200  

[5 rows x 24 columns]

<h3>Compute the cross correlation coefficients on the stocks</h3>
<li>Create a df 'rets' with the one day percentage changes using the column 'close'
<li>Calculate cross correlations on the rets df

In [27]:
c = data['Close']['C'].pct_change()
gs = data['Close']['GS'].pct_change()
jpm = data['Close']['JPM'].pct_change()
ms = data['Close']['MS'].pct_change()
rets = pd.DataFrame(index=ms.index, columns=['C', 'GS', 'JPM', 'MS'])
rets['C'] = c
rets['GS'] = gs
rets['JPM'] = jpm
rets['MS'] = ms
corr_matrix = rets.corr()
corr_matrix

,C,GS,JPM,MS
C,1.000000,0.792099,0.849651,0.810073
GS,0.792099,1.000000,0.813207,0.844273
JPM,0.849651,0.813207,1.000000,0.801133
MS,0.810073,0.844273,0.801133,1.000000


<h3>Select a pair of stocks to trade</h3>
<li>Choose the pair with the highest correlation
<li>You can just eyeball the result and pick the highest correlation pairs
<li>Create a new df 'pairs' with the returns columns of the highest correlation pairs

In [28]:
pairs = pd.DataFrame(index=rets.index, columns=['C', 'JPM'])
pairs['C'] = rets['C']
pairs['JPM'] = rets['JPM']
pairs

,C,JPM
Date,,
2013-01-02,NaN,NaN
2013-01-03,0.003394,-0.002015
2013-01-04,0.025127,0.017725
2013-01-07,0.000943,0.001102
2013-01-08,-0.000236,0.001982
...,...,...
2018-09-17,0.004820,0.002996
2018-09-18,0.006772,0.004041
2018-09-19,0.033072,0.029046


<h3>Calculate the mean and std dev of the columns (returns of the two stocks)</h3>
<li>Note that line 1 contains a NaN. You'll have to get rid of this line
<li>df.describe() returns summary stats for all cols in a dataframe. You can use this to extract means and standard deviations

In [29]:
new_df = rets[['C','JPM']].iloc[1:]
c_std = new_df.describe()['C']['std']
j_std = new_df.describe()['JPM']['std']
c_mean = new_df.describe()['C']['mean']
j_mean = new_df.describe()['JPM']['mean']
print(f"C std: {c_std}, JPM std: {j_std}, C mean: {c_mean}, JPM mean: {j_mean}")

C std: 0.014857630306082118, JPM std: 0.012830319754554335, C mean: 0.0005176123479206794, JPM mean: 0.0007558271790535109


<h3>Create a new column "Long_JPM_Short_C"</h3>
<li>If the return of C is greater than its mean + 0.25* std
<li> AND
<li>If the return of JPM is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [30]:
pairs['Long_JPM_Short_C'] = np.where((pairs['C']>c_mean+0.25*c_std) & (pairs['JPM']<j_mean-0.25*j_std), True, False)
#pairs.groupby('Long_JPM_Short_C').groups
pairs

,C,JPM,Long_JPM_Short_C
Date,,,
2013-01-02,NaN,NaN,False
2013-01-03,0.003394,-0.002015,False
2013-01-04,0.025127,0.017725,False
2013-01-07,0.000943,0.001102,False
2013-01-08,-0.000236,0.001982,False
...,...,...,...
2018-09-17,0.004820,0.002996,False
2018-09-18,0.006772,0.004041,False
2018-09-19,0.033072,0.029046,False


<h3>Create a new column "Long_C_Short_JPM"</h3>
<li>If the return of JPM is greater than its mean + 0.25* std
<li> AND
<li>If the return of C is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [31]:
pairs['Long_C_Short_JPM'] = np.where((pairs['C']<c_mean-0.25*c_std) & (pairs['JPM']>j_mean+0.25*j_std), True, False)
#pairs.groupby('Long_C_Short_JPM').groups
pairs

,C,JPM,Long_JPM_Short_C,Long_C_Short_JPM
Date,,,,
2013-01-02,NaN,NaN,False,False
2013-01-03,0.003394,-0.002015,False,False
2013-01-04,0.025127,0.017725,False,False
2013-01-07,0.000943,0.001102,False,False
2013-01-08,-0.000236,0.001982,False,False
...,...,...,...,...
2018-09-17,0.004820,0.002996,False,False
2018-09-18,0.006772,0.004041,False,False
2018-09-19,0.033072,0.029046,False,False


<h3>Create a new column "Trade_return"</h3>
<li>If Long_JPM_Short_C is True and Long_C_Short_JPM is False, the value of this column should be the return on JPM minus the return on C
<li>If Long_JPM_Short_C is False and Long_C_Short_JPM is True, the value of this column should be the return on C minus the return on JPM
<li>Otherwise the value should be zero

In [32]:
pairs['Trade_return'] = 0
for i in pairs.index:
    if pairs.loc[i,'Long_JPM_Short_C']==True and pairs.loc[i,'Long_C_Short_JPM']==False:
        pairs.loc[i,'Trade_return'] = pairs.loc[i,'JPM'] - pairs.loc[i,'C']
    elif pairs.loc[i,'Long_JPM_Short_C']==False and pairs.loc[i, 'Long_C_Short_JPM']==True:
        pairs.loc[i, 'Trade_return'] = -pairs.loc[i,'JPM'] + pairs.loc[i,'C']
pairs['Trade_return']

Date
2013-01-02    0.0
2013-01-03    0.0
2013-01-04    0.0
2013-01-07    0.0
2013-01-08    0.0
             ... 
2018-09-17    0.0
2018-09-18    0.0
2018-09-19    0.0
2018-09-20    0.0
2018-09-21    0.0
Name: Trade_return, Length: 1442, dtype: float64

<h3>Calculate the sum of this new column</h3>
<li>Are you going to get rich?

In [33]:
somm = pairs['Trade_return'].sum()
somm

-0.6793516982785628

<h3>Calculate the trade_return per year and per month</h3>
<li>Which year has the highest total returns/lowest losses?
<li>Which months in the year did you suffer a loss?
    
<h4> Note: loops will not be accepted

In [44]:
pairs.index = pd.to_datetime(pairs.index)
highest = max(pairs['Trade_return'].resample('Y').sum())
s=pairs['Trade_return'].resample('Y').sum()
#print(s)
index = s.index[s == highest].tolist()
year = index[0].year
print(f"highest: {highest} year: {year}")
s = pairs['Trade_return'].resample('M').sum()
row = s.loc[(s.index.year == year) & (s < 0)]
print(f"Months with loss: {row.index.month.tolist()}")

highest: -0.03546348448723169 year: 2015
Months with loss: [4, 10]
